In [1]:
import pandas
import pymorphy2
import numpy as np
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
import langdetect

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
events_df = pandas.read_csv('meetup_tech_18102020_tz_fixed.csv')

In [3]:
events_df['description'] = events_df['description'].replace(r'https?:\/\/[^\s]+', '', regex=True)
events_df['description'] = events_df['description'].str.lower()
events_df['description'] = events_df['description'].replace('ё', 'е', regex=True)
events_df['description'] = events_df['description'].replace(r'\w*январ\w*|\w*феврал\w*|\w*март\w*|\w*апрел\w*|\w*июн\w*|\w*июл\w*|\w*август\w*|\w*сентябр\w*|\w*октябр\w*|\w*ноябр\w*|\w*декабр\w*|\bма.\b', ' ', regex=True)
events_df['description'] = events_df['description'].replace(r'[^а-яА-Яa-zA-Z]', ' ', regex=True)

In [4]:
events_df['name'] = events_df['name'].str.lower()
events_df['name'] = events_df['name'].replace('ё', 'е', regex=True)
events_df['name'] = events_df['name'].replace(r'\w*январ\w*|\w*феврал\w*|\w*март\w*|\w*апрел\w*|\w*июн\w*|\w*июл\w*|\w*август\w*|\w*сентябр\w*|\w*октябр\w*|\w*ноябр\w*|\w*декабр\w*|\bма.\b', ' ', regex=True)
events_df['name'] = events_df['name'].replace(r'[^а-яА-Яa-zA-Z]', ' ', regex=True)

In [5]:
morph = pymorphy2.MorphAnalyzer()

In [6]:
stopwords = stopwords.words("russian")
stopwords.extend(['что', 'это', 'весь', 'этот', 'привет', 'так', 'вот', 'как', 'ссылка', 'регистрация', 'приглашать', 'еще'])

In [7]:
descr_array = []
names_array = []
for i in range(len(events_df)):
    descr = events_df['description'][i]
    if not pandas.isna(descr):
        descr = [morph.parse(word)[0].normal_form for word in descr.split() if morph.parse(word)[0].tag.POS not in {'INTJ', 'PRCL', 'CONJ', 'PREP', 'NPRO'}
                 and morph.parse(word)[0].normal_form not in stopwords and langdetect.detect(word) == 'ru']
        if not descr:
            descr = np.nan
        else:
            descr = ' '.join(descr)
    descr_array.append(descr)
    name = events_df['name'][i]
    if not pandas.isna(name):
        name = [morph.parse(word)[0].normal_form for word in name.split() if morph.parse(word)[0].tag.POS not in {'INTJ', 'PRCL', 'CONJ', 'PREP', 'NPRO'}
                and morph.parse(word)[0].normal_form not in stopwords]
        name = ' '.join(name)
    names_array.append(name)

events_df['name'] = names_array
events_df['description'] = descr_array

In [8]:
events_df[['name','description']].head()

,name,description
0,google firebase meetup,первый серия год пройти посвятить сборная соля...
1,kotlin hardcore evening,состояться первый который посвятить язык прогр...
2,big android battle languages in java wasteland,питерский разработка состояться альтернативный...
3,mini mobius,скоро случиться главный событие мир российский...
4,i o extended saint petersburg,главное событие каждый разработчик год организ...
